## On plot les results 

In [30]:
from plot_results import  plot_loss_decompose, plot_points
from model import PINNs
from utils import charge_data
import json
import torch
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


In [31]:
folder_name = "3_seemed_working"
epoch = "epoch540"

In [32]:
param_adim = {"V": 1.0, "L": 0.025, "rho": 1.2}

In [23]:
with open("results/" + folder_name + "/hyper_param.json", "r") as file:
    hyper_param = json.load(file)
model = PINNs(hyper_param)
checkpoint = torch.load(
    "results/" + folder_name + "/" + epoch + "/" + "model_weights.pth",
    map_location=torch.device("cpu"),
)
model.load_state_dict(checkpoint["model_state_dict"])
X_train, U_train, X_full, U_full, X_border, X_border_test, mean_std = charge_data(
    hyper_param, param_adim
)

/tmp/ipykernel_19369/2246283635.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(


fichier n°0 chargé
fichier n°1 chargé
fichier n°2 chargé
fichier n°3 chargé
fichier n°4 chargé
Starting X_train
Simu n°0/5
Time:2.795
Simu n°1/5
Time:9.895
Simu n°2/5
Time:16.695
Simu n°3/5
Time:24.188
Simu n°4/5
Time:30.891
X_train OK
X_border OK


In [24]:
import numpy as np

In [33]:
def plot_flow(
    x,
    y,
    t,
    norme_vitesse_data,
    norme_vitesse_predict,
    name_file,
    fps=7,
    title="Norme vitesse",
):
    # Créer une figure et des axes
    # Ajuster la taille de la figure
    fig, ax = plt.subplots(1, 2, figsize=(15, 6))

    # Déterminer les valeurs min et max pour la colormap
    vmin = min(np.min(norme_vitesse_data), np.min(norme_vitesse_predict))
    vmax = max(np.max(norme_vitesse_data), np.max(norme_vitesse_predict))

    # Initialiser les cartes de chaleur
    indices = np.where(t == np.min(t))
    c1 = ax[0].tripcolor(
        x[indices],
        y[indices],
        norme_vitesse_data[indices],
        shading="gouraud",
        cmap="coolwarm",
        vmin=vmin,
        vmax=vmax,
    )
    c2 = ax[1].tripcolor(
        x[indices],
        y[indices],
        norme_vitesse_predict[indices],
        shading="gouraud",
        cmap="coolwarm",
        vmin=vmin,
        vmax=vmax,
    )

    ax[0].set_title("Données")
    ax[0].set_xlabel("x")
    ax[0].set_ylabel("y")

    ax[1].set_title("Prédictions")
    ax[1].set_xlabel("x")
    ax[1].set_ylabel("y")

    # Ajouter une barre de couleur
    cbar = fig.colorbar(c1, ax=ax, orientation="vertical", label=f"{title}")

    # Fonction d'initialisation
    def init():
        return c1, c2

    # Fonction d'animation
    def update(frame):
        print(frame)
        time = list(set(t))
        time.sort()
        indices = np.where(t == time[frame])

        # Mettre à jour la première carte de chaleur
        c1.set_array(norme_vitesse_data[indices].flatten())
        ax[0].set_title("Données")

        # Mettre à jour la deuxième carte de chaleur
        c2.set_array(norme_vitesse_predict[indices].flatten())
        ax[1].set_title("Prédictions")

        # Titre général
        plt.suptitle(f"{title} à t={time[frame]:.2f}", fontsize=16)

        return c1, c2

    # Créer l'animation
    ani = FuncAnimation(
        fig, update, frames=len(set(t)), init_func=init, blit=False, repeat=True
    )
    ani.save(name_file, writer="pillow", fps=fps)


In [ ]:
for ya0_norm in X_full[:, 3].unique():
    masque = X_full[:, 3] == ya0_norm
    X_ = X_full[masque]
    U_ = U_full[masque]
    U_ = U_.detach().numpy()
    predict = model(X_).detach()
    x_norm_pred, y_norm_pred, t_norm_pred = (
        X_.detach().numpy()[:, 0],
        X_.detach().numpy()[:, 1],
        X_.detach().numpy()[:, 2],
    )
    u_norm_pred, v_norm_pred, p_norm_pred = (
        predict.detach().numpy()[:, 0],
        predict.detach().numpy()[:, 1],
        predict.detach().numpy()[:, 2],
    )
    x_pred = (x_norm_pred * mean_std["x_std"].item() +
              mean_std["x_mean"].item()) * param_adim["L"]
    y_pred = (y_norm_pred * mean_std["y_std"].item() +
              mean_std["y_mean"].item()) * param_adim["L"]
    t_pred = (t_norm_pred * mean_std["t_std"].item() + mean_std["t_mean"].item()) * (
        param_adim["L"] / param_adim["V"]
    )
    u_pred = (u_norm_pred * mean_std["u_std"].item() +
              mean_std["u_mean"].item()) * param_adim["V"]
    v_pred = (v_norm_pred * mean_std["v_std"].item() +
              mean_std["v_mean"].item()) * param_adim["V"]
    p_pred = (p_norm_pred * mean_std["p_std"].item() + mean_std["p_mean"].item()) * (
        (param_adim["V"] ** 2) * param_adim["rho"]
    )
    u_data = (U_[:, 0] * mean_std["u_std"].item() +
              mean_std["u_mean"].item()) * param_adim["V"]
    v_data = (U_[:, 1] * mean_std["v_std"].item() +
              mean_std["v_mean"].item()) * param_adim["V"]
    p_data = (U_[:, 2] * mean_std["p_std"].item() + mean_std["p_mean"].item()) * (
        (param_adim["V"] ** 2) * param_adim["rho"]
    )

    norme_vitesse_data = np.sqrt(u_data**2 + v_data**2)
    norme_vitesse_predict = np.sqrt(u_pred**2 + v_pred**2)
    ya0_ = (ya0_norm * mean_std["ya0_std"] + mean_std['ya0_mean']) * param_adim['L']
    plot_flow(
        x_pred,
        y_pred, 
        t_pred, 
        norme_vitesse_data, 
        norme_vitesse_predict, 
        name_file= "results/" + "/" + folder_name + "/" + f"result_vitesse_ya0_{ya0_:.2e}.gif", 
        title = f"Norme vitesse pour ya0={ya0_:.2e}"
        )
    plot_flow(
        x_pred,
        y_pred, 
        t_pred, 
        p_data, 
        p_pred, 
        name_file= "results/" + "/" + folder_name + "/" + f"result_pression_ya0_{ya0_:.2e}.gif", 
        title = f"Norme vitesse pour ya0={ya0_:.2e}"
        )
    

    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
4

In [35]:
u_pred

array([ 0.28244156, -0.0031845 , -0.03463507, ...,  0.98192483,
        0.9754833 ,  0.97556037], shape=(388814,), dtype=float32)

In [36]:
u_data

array([ 0.25939   , -0.00886846, -0.02762461, ...,  0.982116  ,
        0.979859  ,  0.980556  ], shape=(388814,), dtype=float32)

In [37]:
np.mean((u_pred-u_data)**2)

np.float32(0.000117648946)

In [38]:
u_pred.mean()

np.float32(0.8172723)